In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('ab_sem_hw_9.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [4]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


In [5]:
df.version.value_counts()

version
gate_40    45489
gate_30    44700
Name: count, dtype: int64

In [6]:
df.userid.nunique()

90189

In [34]:
df_30 = df.query('version == "gate_30"')
df_40 = df.query('version == "gate_40"')

In [36]:
df_30.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.470000e+04,44700.000000,44700.000000,44700.000000
mean,4.987564e+06,52.456264,0.448188,0.190201
std,2.881026e+06,256.716423,0.497314,0.392464
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.505469e+06,5.000000,0.000000,0.000000
50%,4.983631e+06,17.000000,0.000000,0.000000
75%,7.481497e+06,50.000000,1.000000,0.000000
max,9.999710e+06,49854.000000,1.000000,1.000000


In [10]:
k1 = df_30.retention_1.sum()
k2 = df_40.retention_1.sum()
n1 = df_30.shape[0]
n2 = df_40.shape[0]
k1, k2, n1, n2

(20034, 20119, 44700, 45489)

In [11]:
from statsmodels.stats import proportion

chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), 
                                                   np.array([n1, n2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =3.183, pvalue = 0.074


In [12]:
z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), 
                                                   np.array([n1, n2]))
print('Results are ','z_score =%.3f, pvalue = %.3f'%(z_score, z_pvalue))

Results are  z_score =1.784, pvalue = 0.074


**По результатам оценки AB теста на основе анализа ретеншена одного дня, статистчески значимых различий в версиях нет**

In [13]:
k1_7 = df_30.retention_7.sum()
k2_7 = df_40.retention_7.sum()
k1_7, k2_7, n1, n2

(8502, 8279, 44700, 45489)

In [15]:
from statsmodels.stats import proportion

chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1_7, k2_7]), 
                                                   np.array([n1, n2]))

print('Results are ','chisq =%.3f, pvalue = %.3f'%(chisq, pvalue))

Results are  chisq =10.013, pvalue = 0.002


In [16]:
z_score, z_pvalue = proportion.proportions_ztest(np.array([k1_7, k2_7]), 
                                                   np.array([n1, n2]))
print('Results are ','z_score =%.3f, pvalue = %.3f'%(z_score, z_pvalue))

Results are  z_score =3.164, pvalue = 0.002


**Оба теста показывает, что существует статистически значимое различие в ретеншене в представленных группах, так как pvalue очень мало (значительно менее 5%). Но сразу возникает вопрос в какую сторону мы видим изменения**

In [20]:
df_30_users = k1_7 / n1 * 100
df_40_users = k2_7 / n2 * 100
print(f'% пользователей, который вернулись в течении 7 дней с версией 30 = {df_30_users}')
print(f'% пользователей, который вернулись в течении 7 дней с версией 40 = {df_40_users}')

% пользователей, который вернулись в течении 7 дней с версией 30 = 19.020134228187917
% пользователей, который вернулись в течении 7 дней с версией 40 = 18.200004396667328


**Таким образом, мы видим, что, увеличение кол-ва уровней без сохранения, снизило интерес к игре, и меньший процент пользователей стал возвращаться для повторной игры. Не смотря на то, что разница всего в 1 процент, когда мы говорим о 100 тысяч игроков, 1% - 1000 пользователей.**

Проанализируем еще кол-во согранных партий.

In [21]:
df_30.sum_gamerounds.mean(), df_30.sum_gamerounds.sum()

(52.45626398210291, 2344795)

In [22]:
df_40.sum_gamerounds.mean(), df_40.sum_gamerounds.sum()

(51.29877552814966, 2333530)

In [27]:
df_30.query('sum_gamerounds == 0').shape[0]

1937

In [28]:
df_40.query('sum_gamerounds == 0').shape[0]

2057

Исключим из анализа тех кто не играл

In [29]:
df_30 = df_30.query('sum_gamerounds > 0')
df_40 = df_40.query('sum_gamerounds > 0')

In [33]:
df_30.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.276300e+04,42763.000000,42763.00000,42763.000000
mean,4.993120e+06,54.832332,0.46753,0.198443
std,2.878704e+06,262.217984,0.49895,0.398832
min,1.160000e+02,1.000000,0.00000,0.000000
25%,2.516810e+06,6.000000,0.00000,0.000000
50%,4.993851e+06,18.000000,0.00000,0.000000
75%,7.482403e+06,53.000000,1.00000,0.000000
max,9.999710e+06,49854.000000,1.00000,1.000000


In [46]:
df_40.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.548900e+04,45489.000000,45489.000000,45489.000000
mean,5.009073e+06,51.298776,0.442283,0.182000
std,2.885496e+06,103.294416,0.496663,0.385849
min,3.770000e+02,0.000000,0.000000,0.000000
25%,2.517171e+06,5.000000,0.000000,0.000000
50%,5.007329e+06,16.000000,0.000000,0.000000
75%,7.510762e+06,52.000000,1.000000,0.000000
max,9.999861e+06,2640.000000,1.000000,1.000000


In [40]:
df_30.query('sum_gamerounds > 5000') 

,userid,version,sum_gamerounds,retention_1,retention_7
57702,6390605,gate_30,49854,0,1


**Считаю значение 49854 выбросом и ошибкой в данных, так как данные только а 14 дней, поэтому эту строку исключаю**

In [44]:
df_30 = df_30.query('sum_gamerounds < 5000') 
df_30.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.469900e+04,44699.000000,44699.000000,44699.000000
mean,4.987532e+06,51.342111,0.448198,0.190183
std,2.881051e+06,102.057598,0.497315,0.392450
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.505463e+06,5.000000,0.000000,0.000000
50%,4.983630e+06,17.000000,0.000000,0.000000
75%,7.481522e+06,50.000000,1.000000,0.000000
max,9.999710e+06,2961.000000,1.000000,1.000000


In [55]:
from scipy.stats import kstest, ttest_ind, mannwhitneyu

In [56]:
alpha = 0.05

st = kstest(df_30.sum_gamerounds, 'norm')
print('Distribution is {}normal\n'.format( {True:'not ',
False:''}[st[1] < alpha]));

Distribution is not normal



In [57]:
st = kstest(df_40.sum_gamerounds, 'norm')
print('Distribution is {}normal\n'.format( {True:'not ',
False:''}[st[1] < alpha]));

Distribution is not normal



In [58]:
mw_stats = mannwhitneyu(df_30.sum_gamerounds, df_40.sum_gamerounds)
mw_stats

MannwhitneyuResult(statistic=1024285761.5, pvalue=0.05089155279145376)

При уровне значимости 5%, статистически значемых различий нет, хотя значение прямо на значении уровня значимости. ТАким образом, если говорить, про кол-в игр, которые играют в обоих версиях, то этот вопрос неоднозначный и можно еще посмотреть

**Но если анализировать все результаты вместе, мои рекомендации такие: оставить старую версию с кол-вом уровней 30, так как разница в кол-ве игр была несущественная, как и не существенные различия в ретеншена одного дня. Но если говорить об более долгосрочной связи, ты мы видим стастически значимое изменение (снижение) в ретеншене 7 дней при переходе на версию с 40 уровнями. Долгосрочный эффект более важен на мой взгляд, так как показывает более сильный интерес к игре.**